In [1]:
import bs4 as bs
import datetime as dt
import numpy as np
import os
import pandas as pd
import requests
import talib
import matplotlib.pyplot as plt
import yfinance as yf 
from IPython.display import display
import seaborn as sns
import sys

sys.path.append(r'C:\Users\Jameshuckle\Dropbox\My-Portfolio\AlgorithmicTrading\utils')
from data_util import (save_sp500_tickers)

%matplotlib inline


C:\Users\Jameshuckle\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
data = yf.download(list(save_sp500_tickers().keys()),'2018-08-01','2019-08-8')
data

[*********             19%                       ]  94 of 505 completed

KeyboardInterrupt: 

In [30]:
def atr_threshold_plot(stock, last_month, atr_series, atr):
    pick = pd.DataFrame(last_month["Close"][stock])
    pick['Low'] = last_month["Low"][stock]
    pick['High'] = last_month["High"][stock]
    pick['atr'] = atr_series
    print(pick)
    print(last_month["Close"][stock].mean(),'atr',atr)

def atr_threshold_filter(data, atr_threshold):
    last_month = data.iloc[-61:]
    last_month = last_month.ffill()
    cols = last_month.columns
    data_points = ['Open','High','Low','Close']
    stocks = list(last_month.columns.levels[1])
    atrs = {}
    for stock in stocks:
        #print(stock)
        try:
            atr_series = talib.ATR(
                last_month["High"][stock].values, 
                last_month["Low"][stock].values, 
                last_month["Close"][stock].values, 
                timeperiod=60)

            # ATR is only created after 60 days of data, the first 59 are nans, so drop them
            atr = atr_series[-1]
#             if stock == 'III.L':
#                 atr_threshold_plot(stock, last_month, atr_series, atr)
            # Compare the ATR to the average stock price over the same time to get the expected % move
            atr_percentage = round((atr / last_month["Close"][stock].mean()) * 100,1)
            # weekly atr is approximatly daily ATR * 2
            weekly_atr_percentage = atr_percentage * 2
        except Exception as e:
            print(e)
            weekly_atr_percentage = 0
        weekly_atr_percentage = max(0,weekly_atr_percentage)
        atrs[stock] = weekly_atr_percentage
    atrs = dict(zip(atrs.keys(),atrs.values()))
    keep_stocks = [stock for stock, atr in atrs.items() if atr > atr_threshold]
    return keep_stocks, atrs

# keep_stocks, sorted_atrs = atr_threshold_filter(data, 0.46)
# keep_stocks

In [31]:
def calculate_corrs(close_data, keep_stocks):
    final_data = close_data[keep_stocks]
    raw_corrs = final_data.corr(method='pearson', min_periods=1)
    corrs = raw_corrs.abs()
    return raw_corrs, corrs

#raw_corrs, corrs = calculate_corrs(data, keep_stocks)

In [32]:
def display_correlation_from_list(raw_corrs_data, inspect_list):
    manual_list = raw_corrs.loc[inspect_manual_list,inspect_manual_list]
    display(inspect_list.abs().mean(axis=0).sort_values())
    plt.figure(figsize=(12,8))
    sns.heatmap(inspect_list, annot=True, cmap="PiYG")

inspect_list = ['ALB','LNT','AAL','APC','BR','CHRW','GPS','AIZ','COG']
#display_correlation_from_list(raw_corrs, inspect_list)

In [33]:
def filter_final_correlations(corrs, corr_threshold):
    avg_corrs = list(corrs.mean(axis=0).sort_values().index)
    corrs2 = corrs[avg_corrs].copy()

    already_ranked = []
    dropped = []
    for ticker in avg_corrs: ## loop through in order of least correlated
        already_ranked.append(ticker)
        if ticker in dropped:
            continue
        too_similar = list(corrs2[corrs2[ticker] > corr_threshold].index)
        too_similar = [t for t in too_similar if t not in already_ranked]
        if len(too_similar) > 0:
            corrs2.drop(too_similar, axis=1, inplace=True)
            corrs2.drop(too_similar, axis=0, inplace=True)
            dropped += too_similar

    stock_corr_df = corrs2[list(corrs2.index)]
    
    return stock_corr_df

#stock_df = filter_final_correlations(corrs, corr_threshold = 0.6)

In [34]:
def combine_close_data(list_of_dfs):
    combined = list_of_dfs[0].join(list_of_dfs[1:])
    return combined

In [35]:
def assign_index_to_ticker(indices_names, known_tickers_list, atrs_list, final_tickers_list):
    final_tickers = {}
    for i in range(len(indices_names)):
        index = indices_names[i]
        known_tickers = known_tickers_list[i]
        atrs = atrs_list[i]
        dict_ticker_name = {ticker:name for ticker,name in known_tickers.items() 
                                       if ticker in final_tickers_list}
        sorted_dict_ticker_name = {key:(dict_ticker_name[key],atrs[key]) for key in sorted(dict_ticker_name)}
        final_tickers[index] = sorted_dict_ticker_name
    return final_tickers

In [36]:
# ftse100_tickers = save_ftse100_tickers()
# ftse100_data = yf.download(ftse100_tickers.keys(),'2018-08-01','2019-08-8')
# ftse100_keep_stocks, ftse100_atrs = atr_threshold_filter(ftse100_data, atr_threshold = 4)

#sp500_tickers = save_sp500_tickers()
#sp500_data = yf.download(sp500_tickers.keys(),'2018-08-01','2019-08-8')
#sp500_keep_stocks, sp500_atrs = atr_threshold_filter(sp500_data, atr_threshold = 4)

# russell2000_tickers = save_russell2000_tickers()
# russell2000_data = yf.download(russell2000_tickers.keys(),'2018-08-01','2019-08-8')
# russell2000_data
#russell2000_keep_stocks, russell2000_atrs = atr_threshold_filter(russell2000_data, atr_threshold = 6)

# combined = combine_close_data([ftse100_data.Close, sp500_data.Close])#, russell2000_data.Close])
# raw_corrs, corrs = calculate_corrs(combined, ftse100_keep_stocks + sp500_keep_stocks)# + russell2000_keep_stocks)
# stock_corr_df = filter_final_correlations(corrs, corr_threshold = 0.5)
# display(stock_corr_df)
# assign_index_to_ticker(['ftse100','sp500','russell2000'],
#                              [ftse100_tickers, sp500_tickers, russell2000_tickers],
#                              [ftse100_atrs, sp500_atrs, russell2000_atrs],
#                              list(stock_corr_df.columns))

AttributeError: 'dict_keys' object has no attribute 'replace'